In [ ]:
import pandas as pd
import numpy as np

### Бутстрап

Задача: магазин хочет склонить покупателей к скачиванию мобильного приложения. В качестве эксперимента форму для получения ссылки заменили на кнопку, ведущую в AppStore. На серверах с ID 2 и 3 осталась контрольная версия с формой, на сервере с ID 1 — версия с кнопкой. Проверить, есть ли значимые изменения в средней доле переходов к скачиванию (`VisitPageFlag`)

In [9]:
grocery_data = pd.read_csv("../data/grocery_website.csv")
grocery_data.drop(columns=['LoggedInFlag'])
grocery_data

,RecordID,IP Address,LoggedInFlag,ServerID,VisitPageFlag
0,1,39.13.114.2,1,2,0
1,2,13.3.25.8,1,1,0
2,3,247.8.211.8,1,1,0
3,4,124.8.220.3,0,3,0
4,5,60.10.192.7,0,2,0
...,...,...,...,...,...
184583,184584,114.8.104.1,0,1,0
184584,184585,207.2.110.5,0,2,1
184585,184586,170.13.31.9,0,2,0
184586,184587,195.14.92.3,0,3,0


Некоторые пользователи (идентифицируем их по IP-адресу) заходили на сайт несколько раз, удалите дубликаты пользователей и для каждого заполните `VisitPageFlag=1`, если хотя бы раз для него встречалось значение 1

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Метрика: отношение количества посещений сайта с `VisitPageFlag=1` к общему количеству посещений.<br>
Для обеих групп сгенерируйте бутстрапированное распределение разностей значения метрик для групп A и B. Не забудьте скорректировать значения.

In [ ]:
iterations = 3000
A = grocery_data[grocery_data.ServerID > 1]
B = grocery_data[grocery_data.ServerID == 1]

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Найдите границы доверительного интервала и проверьте, значима ли разница значений метрик для A и B (находится ли 0 за пределами интервала)

In [ ]:
alpha = 0.05

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Вычислите среднее значение скорректированной разности, нам понадобится его запомнить:

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

### Бакетирование

Бутстрапирование на большом объёме данных может происходить очень долго.<br>
* Присвойте каждой записи один из `buckets_count` индексов
* Сгруппируйте в бакеты записи с одинаковыми индексами и группами эксперимента (A/B)
* Для каждого бакета посчитайте сумму `VisitPageFlag` и количество записей

In [ ]:
buckets_count = 5000

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Теперь на полученных выборках повторите процедуру бутстрапа.

In [ ]:
iterations = 3000

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Скорректируйте значения, возьмите среднее и сравните с результатом до бакетизации.<br>

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ